In [1]:
# Parameters
day_obs = '2024-06-26'
instrument = 'latiss'
salindex = 2
telescope = 'AuxTel'
timezone = 'UTC'

<a class="anchor" id="imports"></a>
## Imports and General Setup

In [2]:
import os
from lsst.summit.utils import ConsDbClient
from lsst.summit.utils.utils import computeCcdExposureId
import sqlalchemy
import requests
import pandas as pd

URL = "https://usdf-rsp.slac.stanford.edu/consdb/" # Need to add that part about the headers to client flow through
URL = "http://consdb-pq.consdb:8080/consdb" # Don't use this one

os.environ["no_proxy"] += ",.consdb"

access_token = os.getenv("ACCESS_TOKEN")
headers = {"Authorization": f"Bearer {access_token}"}

sesh = requests.Session()
sesh.headers.update(headers)

%matplotlib inline

In [3]:
try:
    from lsst.summit.utils import ConsDbClient
    have_consdb = True
except ImportError:
    have_consdb = False

if have_consdb:
    client = ConsDbClient(URL)

    print(client.schema())  # list the instruments
    print(client.schema('latiss'))  # list tables for an instrument
    print(client.schema('latiss', 'cdb_latiss.exposure_flexdata')) # specifically flexdata table

['latiss', 'lsstcam', 'lsstcamsim', 'lsstcomcam', 'lsstcomcamsim']
['cdb_latiss.ccdvisit1_quicklook', 'cdb_latiss.ccdexposure', 'cdb_latiss.exposure', 'cdb_latiss.exposure_flexdata', 'cdb_latiss.exposure_flexdata_schema', 'cdb_latiss.visit1_quicklook', 'cdb_latiss.ccdexposure_flexdata', 'cdb_latiss.ccdexposure_flexdata_schema', 'cdb_latiss.ccdexposure_camera']
{'obs_id': ('BIGINT', 'None'), 'key': ('VARCHAR(128)', 'None'), 'value': ('TEXT', 'None')}


<a class="anchor" id="get_records"></a>
## Get Records

In [6]:
# Add consdb
day_obs_int = int(day_obs.replace('-', ''))

visit_query1 = f'''
    SELECT * FROM cdb_{instrument}.visit1
     where day_obs = {day_obs_int}
'''

instrument = 'lsstcomcam'
visit_query2 = f'''
    SELECT * FROM cdb_{instrument}.visit1
     where day_obs = {day_obs_int}
'''

instrument = 'lsstcomcamsim'
visit_query3 = f'''
    SELECT * FROM cdb_{instrument}.visit1
     where day_obs = {day_obs_int}
'''

#what is a quicklook
quicklook_query = f'''
    SELECT q.*  FROM cdb_{instrument}.visit1_quicklook as q,
    cdb_{instrument}.visit1 as v
     WHERE v.day_obs = {day_obs_int} and q.visit_id = v.visit_id
'''

if have_consdb:
    # Use the ConsDB Client, and add a couple of tries
    consdb = ConsDbClient(URL)

    try:
        visits_latiss = consdb.query(visit_query1).to_pandas()
        visits_lsstcc = consdb.query(visit_query2).to_pandas()
        visits_lsstccs = consdb.query(visit_query3).to_pandas()

    except requests.HTTPError or requests.JSONDecodeError:
        # Try twice
        visits_latiss = consdb.query(visit_query1).to_pandas()
        visits_lsstcc = consdb.query(visit_query2).to_pandas()
        visits_lsstccs = consdb.query(visit_query3).to_pandas()

    quicklook = consdb.query(quicklook_query).to_pandas()

else:
    # Assumes at the USDF
    connection = sqlalchemy.create_engine('postgresql://usdf@usdf-summitdb.slac.stanford.edu/exposurelog')
    visits_latiss = pd.read_sql(visit_query1, connection)
    quicklook = pd.read_sql(quicklook_query, connection)

if len(visits_latiss) > 0:
    print(f"Retrieved {len(visits_latiss)} visits from consdb")
    obj_vis = len(visits_latiss.query('img_type == "OBJECT"'))
    print(f"About {obj_vis} of these are object images")

if len(quicklook) > 0:
    visits_latiss = visits_latiss.join(quicklook, on='visit_id', lsuffix='', rsuffix='_q')
    print(f"And added quicklook stats")


Retrieved 786 visits from consdb
About 0 of these are object images
And added quicklook stats


## The schemas in cdb are
`latiss`, `lsstcomcam`, `lsstcomcamsim`

## And each have the following tables
`exposure`, `visit1`

`ccdexposure`, `ccdvisit1`

`ccdexposure_camera`

`visit1_quicklook`, `ccdvisit1_quicklook`

 `exposure_flexdata`, `ccdexposure_flexdata`

 ## We can find more info on the schemas at https://sdm-schemas.lsst.io/

<a class="anchor" id="table"></a>
## Tables of (mostly raw) results

In [16]:
data = pd.DataFrame(visits_latiss)
print(f"Latiss has {len(data.columns)} columns")
print(f"They are {data.columns}")

Latiss has 135 columns
They are Index(['visit_id', 'exposure_name', 'controller', 'day_obs', 'seq_num',
       'physical_filter', 'band', 's_ra', 's_dec', 'sky_rotation',
       ...
       'low_snr_source_count_median', 'low_snr_source_count_total',
       'high_snr_source_count_min', 'high_snr_source_count_max',
       'high_snr_source_count_median', 'high_snr_source_count_total',
       'seeing_zenith_500nm_min', 'seeing_zenith_500nm_max',
       'seeing_zenith_500nm_median', 'n_inputs'],
      dtype='object', length=135)


In [8]:
data

,visit_id,exposure_name,controller,day_obs,seq_num,physical_filter,band,s_ra,s_dec,sky_rotation,...,low_snr_source_count_median,low_snr_source_count_total,high_snr_source_count_min,high_snr_source_count_max,high_snr_source_count_median,high_snr_source_count_total,seeing_zenith_500nm_min,seeing_zenith_500nm_max,seeing_zenith_500nm_median,n_inputs
0,7024062600001,CC_S_20240626_000001,S,20240626,1,i_06,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7024062600002,CC_S_20240626_000002,S,20240626,2,i_06,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7024062600003,CC_S_20240626_000003,S,20240626,3,i_06,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7024062600004,CC_S_20240626_000004,S,20240626,4,i_06,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7024062600005,CC_S_20240626_000005,S,20240626,5,i_06,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
781,7024062600783,CC_S_20240626_000783,S,20240626,783,i_06,None,280.085844,-48.042442,336.97529,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
782,7024062600784,CC_S_20240626_000784,S,20240626,784,i_06,None,280.071854,-48.109481,336.037875,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
783,7024062600785,CC_S_20240626_000785,S,20240626,785,i_06,None,280.241652,-47.902436,335.149192,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
784,7024062600786,CC_S_20240626_000786,S,20240626,786,i_06,None,279.895233,-47.971508,334.430647,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
pd.DataFrame(visits_lsstcc)
#Yes this is empty

""


In [10]:
pd.DataFrame(visits_lsstccs)

,visit_id,exposure_name,controller,day_obs,seq_num,physical_filter,band,s_ra,s_dec,sky_rotation,...,target_name,air_temp,pressure,humidity,wind_speed,wind_dir,dimm_seeing,focus_z,simulated,s_region
0,7024062600001,CC_S_20240626_000001,S,20240626,1,i_06,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,7024062600002,CC_S_20240626_000002,S,20240626,2,i_06,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,7024062600003,CC_S_20240626_000003,S,20240626,3,i_06,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,7024062600004,CC_S_20240626_000004,S,20240626,4,i_06,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,7024062600005,CC_S_20240626_000005,S,20240626,5,i_06,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
781,7024062600783,CC_S_20240626_000783,S,20240626,783,i_06,None,280.085844,-48.042442,336.97529,...,None,None,None,None,None,None,None,0.0,None,None
782,7024062600784,CC_S_20240626_000784,S,20240626,784,i_06,None,280.071854,-48.109481,336.037875,...,None,None,None,None,None,None,None,0.0,None,None
783,7024062600785,CC_S_20240626_000785,S,20240626,785,i_06,None,280.241652,-47.902436,335.149192,...,None,None,None,None,None,None,None,0.0,None,None
784,7024062600786,CC_S_20240626_000786,S,20240626,786,i_06,None,279.895233,-47.971508,334.430647,...,None,None,None,None,None,None,None,0.0,None,None


# Rapid analysis was mentioned
ccdvisit1_quicklook holds some psf* attributes
quicklook might mean rapid analysis

In [14]:
print(f"Many Quicklook columns: {len(quicklook.columns)}")
print(f"They include: {quicklook.columns}")

Many Quicklook columns: 89
They include: Index(['visit_id', 'astrom_offset_mean_min', 'astrom_offset_mean_max',
       'astrom_offset_mean_median', 'astrom_offset_std_min',
       'astrom_offset_std_max', 'astrom_offset_std_median', 'eff_time_min',
       'eff_time_max', 'eff_time_median', 'eff_time_psf_sigma_scale_min',
       'eff_time_psf_sigma_scale_max', 'eff_time_psf_sigma_scale_median',
       'eff_time_sky_bg_scale_min', 'eff_time_sky_bg_scale_max',
       'eff_time_sky_bg_scale_median', 'eff_time_zero_point_scale_min',
       'eff_time_zero_point_scale_max', 'eff_time_zero_point_scale_median',
       'max_dist_to_nearest_psf_min', 'max_dist_to_nearest_psf_max',
       'max_dist_to_nearest_psf_median', 'mean_var_min', 'mean_var_max',
       'mean_var_median', 'n_psf_star_min', 'n_psf_star_max',
       'n_psf_star_median', 'n_psf_star_total', 'psf_area_min', 'psf_area_max',
       'psf_area_median', 'psf_ixx_min', 'psf_ixx_max', 'psf_ixx_median',
       'psf_ixy_min', 'psf_ixy_m

In [12]:
quicklook

,visit_id,astrom_offset_mean_min,astrom_offset_mean_max,astrom_offset_mean_median,astrom_offset_std_min,astrom_offset_std_max,astrom_offset_std_median,eff_time_min,eff_time_max,eff_time_median,...,low_snr_source_count_median,low_snr_source_count_total,high_snr_source_count_min,high_snr_source_count_max,high_snr_source_count_median,high_snr_source_count_total,seeing_zenith_500nm_min,seeing_zenith_500nm_max,seeing_zenith_500nm_median,n_inputs
0,7024062600036,0.029990,0.029990,0.029990,0.014731,0.014731,0.014731,6.980644,6.980644,6.980644,...,None,None,None,None,None,None,None,None,None,9
1,7024062600039,0.012897,0.012897,0.012897,0.007223,0.007223,0.007223,8.623010,8.623010,8.623010,...,None,None,None,None,None,None,None,None,None,9
2,7024062600042,0.009001,0.009001,0.009001,0.005302,0.005302,0.005302,9.158998,9.158998,9.158998,...,None,None,None,None,None,None,None,None,None,9
3,7024062600045,0.013785,0.013785,0.013785,0.007479,0.007479,0.007479,9.575220,9.575220,9.575220,...,None,None,None,None,None,None,None,None,None,9
4,7024062600048,0.015490,0.015490,0.015490,0.008647,0.008647,0.008647,7.328437,7.328437,7.328437,...,None,None,None,None,None,None,None,None,None,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
690,7024062600778,0.008086,0.008086,0.008086,0.004840,0.004840,0.004840,10.270951,10.270951,10.270951,...,None,None,None,None,None,None,None,None,None,9
691,7024062600780,0.010005,0.010005,0.010005,0.005335,0.005335,0.005335,10.552485,10.552485,10.552485,...,None,None,None,None,None,None,None,None,None,9
692,7024062600781,0.009080,0.009080,0.009080,0.005170,0.005170,0.005170,10.516340,10.516340,10.516340,...,None,None,None,None,None,None,None,None,None,9
693,7024062600782,0.023023,0.023023,0.023023,0.011832,0.011832,0.011832,11.082850,11.082850,11.082850,...,None,None,None,None,None,None,None,None,None,9
